In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [ ]:
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1) 
g = sns.countplot(Y_train)
Y_train.value_counts()

In [ ]:
X_train.isnull().any().describe()

In [ ]:
test.isnull().any().describe()

In [ ]:
X_train = X_train / 255.0
test = test / 255.0

In [ ]:
'''Grayscale normalization to reduce the effect of illumination's differences.
Also, CNN converges faster on [0..1] data than on [0..255].'''

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [ ]:
'''-1 means that the new shape should be compatible with the original shape.
If we want to reshape (5,5,8) to (10,10,2), we can also write (10,10,-1).
The third dimension will be calculated.'''

In [ ]:
Y_train = to_categorical(Y_train, num_classes = 10)
#one-hot encoding

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

In [ ]:
sns.set(style='white', context='notebook', palette='deep')

In [ ]:
g = plt.imshow(X_train[0][:,:,0])

In [ ]:
'''[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out'''

In [ ]:
'''For padding :
    SAME: output size is the same as input size(for stride 1).
    VALID: Don't apply any padding'''

In [ ]:
'''pool_size: integer or tuple of 2 integers, factors by which to downscale (vertical, horizontal). 
If only one integer is specified, the same window length will be used for both dimensions.
strides: Integer, tuple of 2 integers, or None. Strides values. If None, it will default to pool_size.'''

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(AvgPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(AvgPool2D(pool_size=(2,2)))

In [ ]:
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(AvgPool2D(pool_size=(2,2)))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(AvgPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

In [ ]:
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
'''The most important function is the optimizer. This function will iteratively improve parameters 
(filters kernel values, weights and bias of neurons ...) in order to minimise the loss.'''

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [ ]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
'''In order to make the optimizer converge faster and closest to the global minimum 
of the loss function, we will use an annealing method of the learning rate (LR).'''

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
'''With the ReduceLROnPlateau function from Keras.callbacks, we choose to 
reduce the LR by half if the accuracy is not improved after 3 epochs.'''

In [ ]:
epochs = 30
batch_size = 64

In [ ]:
history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs, 
          validation_data = (X_val, Y_val),callbacks=[learning_rate_reduction])

In [ ]:
results = model.predict(test)

In [ ]:
results = np.argmax(results,axis = 1)
# select the index with the maximum probability

In [ ]:
results = pd.Series(results,name="Label")

In [ ]:
len(results)

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

In [ ]:
submission.to_csv("SM.csv",index=False)